In [29]:
import math
import os
from PIL import Image, ImageFont, ImageDraw

In [30]:
# Configuration
# macOS 下路径写法可能需要调整

# 准备生成的字体信息
CELL_SIZE = 18      # 每个容纳文字的格子大小
FONT_SIZE = 16      # 文字目标大小
FONT_FILEPATH = 'CabinetGrotesk-Thin.ttf'    # 字体文件路径

OUTPUT_PLAYDATE_FONT_NAME = 'CabinetGrotesk-Thin-16'   # 生成的 Playdate 字体名字
OUTPUT_DIR = 'output'    # 输出文件夹路径

In [31]:
# 高级设置，一般无需更改

# 需要生成的字表
# character_table 下默认为三个等级的通用规范汉字表，可根据所需增删。
TEXT_TABLE_FILEPATH = []

text_table_characters = ""
for filepath in TEXT_TABLE_FILEPATH:
    with open(filepath, encoding="utf-8") as f:
        lines = [line.strip() for line in f]
    text_table_characters += "".join(lines)

TEXT_LATIN_SET = " !\"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~…¥‼™©®"
TEXT_PUNCTUATION = ""
TEXT_JAPANESE_KANA = ""

TEXT_TABLE_STR = TEXT_LATIN_SET + TEXT_PUNCTUATION + TEXT_JAPANESE_KANA + text_table_characters + "�"

IMAGE_TABLE_CELL_WIDTH_CNT = math.isqrt(len(TEXT_TABLE_STR))

_, FONT_Y_OFFSET, _, _ = ImageFont.truetype(FONT_FILEPATH, FONT_SIZE).getbbox("啊")   # （自动计算）字体垂直坐标的偏移值
FONT_Y_OFFSET = 2                                                                      # （手动设置）字体垂直坐标的偏移值
# print(f"{FONT_Y_OFFSET=}")

In [32]:
def create_image(text=TEXT_TABLE_STR, font_file=FONT_FILEPATH, cell_size=CELL_SIZE, font_size=FONT_SIZE, font_y_offset=FONT_Y_OFFSET, width_cell_cnt=IMAGE_TABLE_CELL_WIDTH_CNT):
    # 加载字体文件
    font = ImageFont.truetype(font_file, font_size)
  
    # 计算图像大小
    width = width_cell_cnt * cell_size
    height = ((len(text) - 1) // width_cell_cnt + 1) * cell_size
    print(f"{width=}, {height=}")

    # 创建透明图像
    image = Image.new("RGBA", (width, height), (0, 0, 0, 0))
    draw = ImageDraw.Draw(image)

    # 渲染每个字符
    for i, char in enumerate(text):
        x = (i % width_cell_cnt) * cell_size
        y = (i // width_cell_cnt) * cell_size
        pos = (x, y-font_y_offset)
        draw.text(pos, char, font=font, fill=(0, 0, 0, 255))
    
    return image


def get_font_width(font_file, font_size, text):
    font = ImageFont.truetype(font_file, font_size)
    # left, top, right, bottom = font.getbbox(text)
    width = font.getlength(text)
    return int(width)


def generate_fnt_table(text=TEXT_TABLE_STR):
    font_table = []
    for i, char in enumerate(text):
        font_width = get_font_width(font_file=FONT_FILEPATH, font_size=FONT_SIZE, text=char)
        if char == " ":
            char = "space"
        font_table.append(f"{char}		{font_width}")
    
    with open(os.path.join(OUTPUT_DIR, f"{OUTPUT_PLAYDATE_FONT_NAME}.fnt"), 'w', encoding='utf-8') as file:
        file.write('\n'.join(font_table))

In [33]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

img = create_image()
img.save(os.path.join(OUTPUT_DIR, f"{OUTPUT_PLAYDATE_FONT_NAME}-table-{CELL_SIZE}-{CELL_SIZE}.png"))
generate_fnt_table()

width=180, height=198


: 